In [ ]:
# @title 🛠️ Rex2Fusion_Linklist
# @markdown Enter your settings and click the **Play** (▶️) button to the left.

import time
import pandas as pd
import re
import os
from google.colab import files
import io

# @markdown ### Settings
Linklist_Start_Number = 50 # @param {type:"integer"}

print("Waiting for file upload...")

# 1. Upload
uploaded = files.upload()

if not uploaded:
    print("❌ No file uploaded.")
else:
    start_time = time.time()
    filename = next(iter(uploaded))
    print(f"\nProcessing: {filename}...")

    # --- ROBUST LOGIC ---
    # Read and clean file
    raw_data = uploaded[filename]
    try:
        content = raw_data.decode('utf-8-sig', errors='ignore')
    except:
        content = raw_data.decode('latin-1', errors='ignore')
    content = content.replace('\x00', '')
    lines = content.splitlines()

    # Setup
    current_id = Linklist_Start_Number
    collections = []
    unit_map = {}

    # Fast Regex Patterns
    name_pattern = re.compile(r"^#([^;]+)")
    code_pattern = re.compile(r"#([^\[;#]+)\[([\d\.,]+)\]")

    # Parsing
    for line in lines:
        line = line.strip()
        if not line: continue

        if line.startswith("#"):
            # Collection Name
            name_match = name_pattern.search(line)
            current_coll_name = name_match.group(1).strip() if name_match else line.split(";")[0].replace("#", "").strip()

            # Find Codes
            found_codes = code_pattern.findall(line)
            for code, qty_str in found_codes:
                qty_str = qty_str.replace(",", ".")
                try:
                    qty = float(qty_str)
                    loop_count = int(qty) if qty.is_integer() else 1
                except:
                    loop_count = 1

                for _ in range(loop_count):
                    collections.append({
                        "Linklist ID": current_id,
                        "Linklist Name": current_coll_name,
                        "Linklist Units": code.strip(),
                        "Number of Units": "[1.000]",
                        "Series ID": 0
                    })
            current_id += 1

        elif line.startswith(";#"):
            # Unit Name
            unit_name = line.replace(";", "").replace("#", "").strip()
            if unit_name:
                active_id = current_id - 1
                if unit_name not in unit_map: unit_map[unit_name] = []
                unit_map[unit_name].append(active_id)

    # Build Excel
    df = pd.DataFrame(collections)
    req_cols = ["Linklist ID", "Linklist Name", "Linklist Units", "Number of Units", "Series ID"]
    for c in req_cols:
        if c not in df.columns: df[c] = ""

    unit_data_list = []
    for u_name, id_list in unit_map.items():
        link_str = "<"
        for idx, linked_id in enumerate(id_list):
            link_str += f"{idx+1}~1~{linked_id}|"
        link_str += ">"
        unit_data_list.append({"Unit Name": u_name, "Linked Lists": link_str})

    df_units = pd.DataFrame(unit_data_list)
    final_df = pd.concat([df.reset_index(drop=True), df_units.reset_index(drop=True)], axis=1)

    final_cols = ["Linklist ID", "Linklist Name", "Linklist Units", "Number of Units", "Series ID", "Unit Name", "Linked Lists"]
    for c in final_cols:
        if c not in final_df.columns: final_df[c] = ""
    final_df = final_df[final_cols].fillna("")

    # Download
    output_filename = os.path.splitext(filename)[0] + "_FixedOutput.xlsx"
    final_df.to_excel(output_filename, index=False)

    print(f"✅ Success! File generated in {round(time.time() - start_time, 2)}s")
    files.download(output_filename)